# Writing all useful output from the simulations into one netcCDF archive

## General architecture

For each simulations (F05H05, etc), we need the following:
* Variables: 
    1. horizontally averaged integrand of the flux $\Phi$ for the upstream:
        $$\phi_{U}(z, t) = \frac{1}{\mu}\frac{g}{\rho_1}\int_{U}\frac{|\nabla b|^2}{\partial_z\rho_*}\frac{\text d x}{L_U(z)},$$
        where I did *not* average in time, and where $L_{U}(x)$ is the length of the integration path at a given depth, which depends on $z$ because of the topography
    2. Vertical sorted buyoancy gradient in the upstream $N_{*, U}^2(z, t)$. Note that as we discussed, it should *not* depend on $x$, ever. If it does, then the sorted buoyancy needs to be binned in $x$ to match the dimension of $z$, I believe. Note that I'm not 100% sure how it should be done because I've never done it myself, but it should not be a function of $x$.
    2. $L_U(z)$, because we will probably need it later
    3. $\phi_D(z, t)$, same as $\phi_U$ but for downstream 
    4. $N_{*, D}(z, t)$, same as $N_{*, U}(z, t)$ but for downstream 
    5. $L_D(z)$, same as $L_U(z)$ but for downstream
    6. Time
    7. $z$ (same for every simulation but it is simpler that way)
* Dimensions:
    1. Time
    2. $z$
    
Each simulation's data will be in a group that is named after the simulations name (e.g., `F05H09`).

## Preparing Data

The lines below are me, creating fake data to make the procedure work. You do not need to format your data in the same way, you just need to have the right inputs for the `create_group` function below.

In [1]:
import numpy as np

etas = [0.5, 0.95, 1.2, 2.]  # list of non-dim drafts
Frs = [0.5, 1., 1.5, 2.]  # list of Froude numbers

# Create empty variables, later to be filled with simulation data 
simIDs = []  # empty list that will contain all simulation names
# Dictionaries: the key will be e.g. F05H09, filled with corresponding data
time = {}  # empty dictionnary for the time array of each simulation
LUs = {}
LDs = {}
pUs = {}
pDs = {}
NUs = {}
NDs = {}

Below, I create a set of fake data. I will use the same for each simulation, but in the final version, you would load the simulations' values.

In [2]:
rho1 = 1000  # [kg/m3] or whatever the actual value is
g = 9.81  # [m/s2] idem
mu = 2e-3  # [m2/s] salt diffusivity

depth = np.linspace(0., 80., 640)  # fake z array; 
# The values might be correct but use the simulation's array anyway
FakeTime = np.linspace(0., 132., 300)
FakeLU = 1*depth  # To have the same size as z
FakeLD = 1*depth
FakephiU = np.meshgrid(FakeTime, depth)  # again to have correct size
FakephiD = np.meshgrid(FakeTime, depth)  # again to have correct size
FakeNU = np.meshgrid(FakeTime, depth)  # again to have correct size
FakeND = np.meshgrid(FakeTime, depth)  # again to have correct size

for H in etas:
    for F in Frs:
        simID = "F{0:02d}H{1:02d}".format(int(F*10), int(H*10))  # F05H05, etc
        simIDs.append(simID)
        time[simID] = FakeTime
        LUs[simID] = FakeLU
        LDs[simID] = FakeLD
        pUs[simID] = FakeLU
        pDs[simID] = FakeLD
        NUs[simID] = FakeLU + 1. # +1 because I don't want zeros
        NDs[simID] = FakeLD + 1. 

## Wrapping the data into a netCDF archive

Once you replaced the cell above with one that pupulates the dictionaries with actual data, you should be able to use the rest of the code as is.

In [3]:
from create_ice_keels_netcdf import *
import os

if os.path.exists("ice-keels.nc"):  # we start from scratch
    os.remove("ice-keels.nc")
else:
    print("The file does not exist")

create_netcdf(simIDs, time, depth, LUs, LDs, pUs, pDs, NUs, NDs)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: Mixing under ice keels
    dimensions(sizes): 
    variables(dimensions): 
    groups: F05H05, F10H05, F15H05, F20H05, F05H09, F10H09, F15H09, F20H09, F05H12, F10H12, F15H12, F20H12, F05H20, F10H20, F15H20, F20H20
